In [ ]:
import pdg
import pandas as pd
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
import os

# point matplotlib ticks inwards
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
# add top and right ticks
plt.rcParams['axes.spines.top'] = True
plt.rcParams['axes.spines.right'] = True
# use tex
plt.rcParams['text.usetex'] = True

# Get PDG 2025 data


In [ ]:
api = pdg.connect('sqlite:///data/pdgall-2025-v0.2.0.sqlite')

con = sqlite3.connect('data/pdgall-2025-v0.2.0.sqlite')
cur = con.cursor()
command = """
SELECT pdgid.description, pdgmeasurement.pdgid, pdgdata.value_type, pdgdata.in_summary_table, pdgdata.value, pdgmeasurement_values.value, pdgmeasurement_values.error_positive, pdgmeasurement_values.error_negative, pdgmeasurement_values.stat_error_positive, pdgmeasurement_values.stat_error_negative
FROM pdgmeasurement_values
     JOIN pdgmeasurement ON pdgmeasurement.id = pdgmeasurement_values.pdgmeasurement_id
     JOIN pdgid ON pdgid.id = pdgmeasurement.pdgid_id
     JOIN pdgdata ON pdgdata.pdgid_id = pdgid.id
--     JOIN pdgparticle ON pdgparticle.pdgid = pdgid.parent_pdgid
WHERE pdgmeasurement_values.used_in_average AND pdgmeasurement_values.value IS NOT NULL AND pdgdata.edition = '2025' AND pdgdata.value_type = 'AC'
"""
res = cur.execute(command)
data = res.fetchall() #WHERE 
columns = [col[0] for col in res.description]
print(len(data), 'measurements')
print(columns)

In [ ]:
df = pd.DataFrame(data, columns=['pdgid.description', 'pdgid', 'type', 'insummary', 'avg', 'measurement', 'error_positive', 'error_negative', 'stat_error_positive', 'stat_error_negative'])
df['error'] = (df['error_positive'] + df['error_negative'])/2
df['staterr'] = (df['stat_error_positive'] + df['stat_error_negative'])/2
# replace NaN staterr with total error
staterr = np.array(df['staterr'])
staterr[np.isnan(staterr)] = np.array(df['error'])[np.isnan(staterr)]
# replace zero staterr with smallest nonzero staterr

# see https://pdg.lbl.gov/encoder_listings/s035.pdf, S035C19 LEE, 0.003% star. error * 8.42
staterr[(staterr == 0) & np.array(df['pdgid'] == 'S035C19')] = 0.0002526

# staterr[staterr == 0] = np.min(staterr[staterr > 0])/2
df['staterr'] = staterr



df['std_resid'] = (df['measurement'] - df['avg']) / df['error']
# only keep rows where there are at least 3 measurements
df = df.groupby('pdgid').filter(lambda x: len(x) >= 3)
print('Number of properties:', len(df['pdgid'].unique()))
print('Number of measurements:', len(df))

df['value'] = df['measurement']
# to_drop = ['measurement', 'error_positive', 'error_negative', 'stat_error_positive', 'stat_error_negative']
# df = df.drop(columns=to_drop)

pdg2025_stat = df.copy()
pdg2025_stat['uncertainty'] = pdg2025_stat['staterr']
del pdg2025_stat['staterr'], pdg2025_stat['error']
df_gb = pdg2025_stat.groupby('pdgid', group_keys=False)
pdg2025_stat_dfs = [df_gb.get_group(x).copy() for x in df_gb.groups]

pdg2025_both = df.copy()
pdg2025_both['uncertainty'] = pdg2025_both['error']
del pdg2025_both['staterr'], pdg2025_both['error']
df_gb = pdg2025_both.groupby('pdgid', group_keys=False)
pdg2025_both_dfs = [df_gb.get_group(x).copy() for x in df_gb.groups]

# Load other datasets

In [ ]:
def dfs_from_folder(folder):
    path_list = os.listdir(folder)
    files = [path for path in path_list if path.endswith('.csv')]
    files.sort()
    dfs = [pd.read_csv(f'{folder}/{file}', comment='#') for file in files]
    return dfs, files
manylabs2_dfs, manylabs2_files = dfs_from_folder('data/psymetadata')
bipm_radionuclide_dfs, bipm_radionuclide_files = dfs_from_folder('data/bipm-radionuclide')
baker_medical_dfs, baker_medical_files = dfs_from_folder('data/baker-medical/clean')
for df in baker_medical_dfs:
    df['uncertainty'] = np.sqrt(df['sigma2'])
baker_pdg2011_both_dfs, baker_pdg2011_both_files = dfs_from_folder('data/baker-pdg2011-both')
baker_pdg2011_stat_dfs, baker_pdg2011_stat_files = dfs_from_folder('data/baker-pdg2011-stat')

datasets = {
    'pdg2025-stat': pdg2025_stat_dfs,
    'pdg2025-both': pdg2025_both_dfs,
    'baker-pdg2011-stat': baker_pdg2011_stat_dfs,
    'baker-pdg2011-both': baker_pdg2011_both_dfs,
    'bipm-radionuclide': bipm_radionuclide_dfs,
    'manylabs2': manylabs2_dfs,
    'baker-medical': baker_medical_dfs,
}
files = {
    'baker-pdg2011-stat': baker_pdg2011_stat_files,
    'baker-pdg2011-both': baker_pdg2011_both_files,
    'bipm-radionuclide': bipm_radionuclide_files,
    'manylabs2': manylabs2_files,
    'baker-medical': baker_medical_files,
}
nice_names = {
    'pdg2025-stat': 'PDG 2025 (stat)',
    'pdg2025-both': 'PDG 2025 (stat + syst)',
    'baker-pdg2011-stat': 'Baker 2013: PDG 2011 (stat)',
    'baker-pdg2011-both': 'Baker 2013: PDG 2011 (stat + syst)',
    'bipm-radionuclide': 'BIPM Radionuclide',
    'manylabs2': 'Many Labs 2',
    'baker-medical': 'Baker 2013: medical',
}
nice_names_break = {
    'pdg2025-stat': 'PDG 2025 (stat)',
    'pdg2025-both': 'PDG 2025 (stat + syst)',
    'baker-pdg2011-stat': 'Baker 2013\nPDG 2011 (stat)',
    'baker-pdg2011-both': 'Baker 2013\nPDG 2011 (stat + syst)',
    'bipm-radionuclide': 'BIPM Radionuclide',
    'manylabs2': 'Many Labs 2',
    'baker-medical': 'Baker 2013\nmedical',
}

In [ ]:
# for each pdgid, do some operations on each row with that pdgid
def process_group(name, df):

    sigma = np.array(df['uncertainty'])
    sigma2 = sigma**2
    if not np.all(sigma2 > 0):
        print(name)
        print(df)

    S = np.sum(1/sigma2)

    Xbar = np.sum(df['value'] / sigma2) / S
    std = np.sqrt(sigma2*(1-1/(sigma2*S))**2 + (S-1/sigma2)/(S**2))
    df['std_resid_adj'] = (df['value'] - Xbar) / std

from methods import I2
I2s = {}
for name, dataset in datasets.items():
    I2s[name] = []
    for df in dataset:
        process_group(name, df)
        I2s[name].append(I2(df['value'], df['uncertainty']))

In [ ]:
from scipy.stats import norm
x = np.linspace(0, 5, 100)


# plt.plot(x, norm.pdf(x, 0, 1), color='red', label='Standard Normal PDF')
# plt.title('Standardized residuals of PDG measurements')
# plt.legend(frameon=False)
# plt.savefig('figs/pdg_std_residuals.pdf', bbox_inches='tight')
# plt.show()

fig, axs = plt.subplots(2, 4, figsize=(8, 4), sharex=True, sharey=True, gridspec_kw={'hspace': 0, 'wspace': 0})
n_datasets = len(datasets)
for i, (name, dataset) in enumerate(datasets.items()):
    ax = axs.flatten()[i]
    std_resids = []
    for df in dataset:
        std_resids += list(df['std_resid_adj'])
    std_resids = np.abs(std_resids)
    ax.hist(std_resids, bins=30, range=(0, 5), density=True, color='grey', label='Standardized residuals')
    ax.axvline(0, color='black', linestyle='--')
    ax.plot(x, norm.pdf(x, 0, 1)*2, color='red', label='Standard Normal PDF')
    ax.set_xlim(0, 4.5)
    ax.set_ylim(0, 1.1)
    # put title inside axis in top right corner
    #ax.set_title(nice_names[name])
    ax.text(0.95, 0.95, nice_names_break[name], transform=ax.transAxes, ha='right', va='top')
    # add top and right ticks
    ax.tick_params(top=True, right=True)
    ax.set_xlabel('Standardized residuals')
    ax.set_ylabel('Density')
    # remove y tick labels if axis is on the right
    # if i % 2 == 1:
    #     ax.set_yticklabels([])


empty_axs = axs.flatten()[i+1:]
for ax in empty_axs:
    ax.axis('off')

plt.suptitle('Standardized residuals of study results with no ground truth')
plt.tight_layout()
plt.savefig('figs/noground-std-resids.pdf', bbox_inches='tight')
plt.show()

In [ ]:
from methods import birge, random_effects_mle, I2
from scipy.stats import norm
from tqdm import tqdm
from collections import defaultdict
brs = defaultdict(list)
taus = defaultdict(list)
I2s = defaultdict(list)
errscale_ps = defaultdict(list)
brs_cont = defaultdict(list)
taus_cont = defaultdict(list)
I2s_cont = defaultdict(list)
errscale_ps_cont = defaultdict(list)

birge_loglikes = defaultdict(list)
re_loglikes = defaultdict(list)
fe_loglikes = defaultdict(list)
ns = defaultdict(list)
exponent_loglikes = defaultdict(list)
exponent_grid = np.linspace(0, 1, 101, endpoint=True)
# print(exponent_grid)

pdg2025bad = ['M047R7', 'M002R19', 'M049R52', 'M055R6', 'M053R02', 'M052R4', 'M056R4', 'M057R4', 'M070R24', 'M070R50', 'M070R60', 'M070R7', 'M070R82', 'M070R83', 'M070R84', 'M070R86', 'M070R87','M070R9', 'M070S6', 'M071R22', 'M071R28','M071S10', 'S040R11', 'S041B24', 'S041B41', 'S041C5', 'S041R3', 'S041R39', 'S041R90', 'S041S47', 'S041R65', 'S041S50', 'S041T03', 'S042B26', 'S042B27', 'S042B43', 'S042B47', 'S042B58', 'S042P59', 'S042R2', 'S042R20', 'S042R22', 'S042R23', 'S042R3', 'S042R47', 'S042R48', 'S042S24', 'S042S59', 'S049R21', 'S049S7', 'S049R24', 'S042S88', 'S086R3', 'S086R33', 'S086R32', 'S086R8', 'S086R34', 'S086R6']

names = defaultdict(list)

for name, dfs in datasets.items():
    # if name not in ['baker-pdg2011-stat', 'baker-medical', 'baker-pdg2011-both']:
    #     continue
    print(name)
    for i, df in tqdm(enumerate(dfs), total=len(dfs)):
        
        if name in ['pdg2025-stat', 'pdg2025-both']:
            if df['pdgid'].iloc[0] in pdg2025bad:
                continue
            names[name].append(df['pdgid'].iloc[0])
        else:
            names[name].append(files[name][i])

        values = np.array(df['value'])
        sigmas = np.array(df['uncertainty'])

        scaler = np.std(values)
        if scaler == 0:
            continue
        values = values / scaler
        sigmas = sigmas / scaler

        # sigmas = sigmas/np.mean(sigmas)
        _, muhat_birge, _, chat = birge(values, sigmas, coverage=0.6827, mle=True)
        brs[name].append(chat)
        mean_sigma = np.mean(sigmas)

        _, muhat_re, _, tau = random_effects_mle(values, sigmas, coverage=0.6827)
        taus[name].append(np.mean(tau/sigmas))
        I2s[name].append(I2(values, sigmas))
        # print(files[name][i], len(values), np.round(I2s[name][-1], 2))
        # if I2s[name][-1] <= 0:
        #     continue
        # if files[name][i] == 'RHO770.csv':
        #     continue

        tau_grid = np.concatenate((np.linspace(0,1.1,100)*np.std(values), np.logspace(-4,4,100)))
        
        # theta_grid = np.concatenate((np.linspace(muhat_birge, muhat_re, 100), np.linspace(np.min(values), np.max(values), 100)))
        exponent_loglike = np.zeros(len(exponent_grid))
        for i,exponent in enumerate(exponent_grid):
            # scale = np.sqrt(sigmas[:,np.newaxis,np.newaxis]**2 + ((tau_grid)**2) * (sigmas[:,np.newaxis,np.newaxis]**(2*exponent)))
            scale = np.sqrt(sigmas[:,np.newaxis]**2 + ((tau_grid)**2) * (sigmas[:,np.newaxis]**(2*exponent)))
            
            w = 1/scale**2
            assert w.shape == (len(values), len(tau_grid))
            # print(np.sum(w * values, axis=0).shape)
            theta_mle = np.sum(w * values[:,np.newaxis], axis=0) / np.sum(w, axis=0)
            assert theta_mle.shape == tau_grid.shape

            # print(scale.shape)
            loglike = norm.logpdf(values[:,np.newaxis], loc=theta_mle, scale=scale)
            assert loglike.shape == (len(values), len(tau_grid))
            exponent_loglike[i] = np.max(np.sum(loglike, axis=0))
        exponent_loglikes[name].append(exponent_loglike)


        # for i in range(10):
        #     # generate values with same sigmas but no unaccounted for errors.
        #     # to be used as a control when analyzing the distribution of chat and tau
        #     values_control = np.random.normal(loc=0, scale=sigmas)
        #     _, _, _, chat_cont = birge(values_control, sigmas, coverage=0.6827)
        #     brs_cont[name].append(chat_cont)
        #     _, _, _, tau_cont = random_effects_mle(values_control, sigmas, coverage=0.6827)
        #     taus_cont[name].append(np.mean(tau_cont/sigmas))
        #     I2s_cont[name].append(I2(values_control, sigmas))



        birge_loglikes[name].append(np.sum(norm.logpdf(values, loc=muhat_birge, scale=sigmas*chat)))
        if any(np.array(birge_loglikes[name])==-np.inf):
            print(i)
            break

        re_loglikes[name].append(np.sum(norm.logpdf(values, loc=muhat_re, scale=np.sqrt(sigmas**2+tau**2))))
        fe_loglikes[name].append(np.sum(norm.logpdf(values, loc=muhat_birge, scale=sigmas)))
        ns[name].append(len(df))

    birge_loglikes[name] = np.array(birge_loglikes[name])
    re_loglikes[name] = np.array(re_loglikes[name])
    fe_loglikes[name] = np.array(fe_loglikes[name])
    ns[name] = np.array(ns[name])

In [ ]:
print([(k, np.sum(n)) for k, n in ns.items()])

In [ ]:
for name in exponent_loglikes.keys():
    assert len(exponent_loglikes[name]) == len(birge_loglikes[name])

In [ ]:
# _, theta_mle, _, tau_mle = random_effects_mle(values, sigmas, coverage=0.6827)
# theta_mle, tau_mle

In [ ]:
# theta_idx, tau_idx = np.unravel_index(np.argmax(loglike), loglike.shape)
# theta_grid[theta_idx], tau_grid[tau_idx]

In [ ]:
# print(np.sum(norm.logpdf(values, loc=theta_mle, scale=np.sqrt(sigmas**2+tau_mle**2))))
# print(np.sum(norm.logpdf(values, loc=theta_grid[theta_idx], scale=np.sqrt(sigmas**2+tau_grid[tau_idx]**2))))

In [ ]:
from scipy.stats import chi2

best_exponents = {}
exponent_cis = {}
for name, exponent_loglike in exponent_loglikes.items():
    exponent_loglike = np.sum(exponent_loglike, axis=0)
    best_exponents[name] = exponent_grid[np.argmax(exponent_loglike)]
    logratio = 2 * (np.max(exponent_loglike) - exponent_loglike)
    within = logratio < chi2.ppf(0.95, df=1)
    idx_l = np.argmax(within)
    idx_u = -np.argmax(within[::-1]) - 1
    low = exponent_grid[idx_l]
    high = exponent_grid[idx_u]
    exponent_cis[name] = [low, high]

In [ ]:
exponent_cis

In [ ]:
best_exponents

In [ ]:
rows = []
for name in best_exponents.keys():
    rows.append([r'\texttt{' + name + r'}', f'${best_exponents[name]}$', f'$[{exponent_cis[name][0]}, {exponent_cis[name][1]:.2f}]$'])
    
# use np to save latex table with & separator
txt = ' \\\\\n'.join([' & '.join(row) for row in rows])
print(txt)
with open('tables/noground-bakermodel.tex', 'w') as f:
    f.write(txt)

In [ ]:
# plt.plot(exponent_grid, exponent_loglikes['baker-pdg2011-both'][4])
# for exponent_loglike in exponent_loglikes['baker-pdg2011-stat']:
    # plt.plot(exponent_grid, (exponent_loglike - np.max(exponent_loglike))/(np.max(exponent_loglike)-np.min(exponent_loglike)))
which = 'baker-pdg2011-stat'
for name, exponent_loglike in exponent_loglikes.items():
    exponent_loglike = np.sum(np.array(exponent_loglike), axis=0)
    plt.plot(exponent_grid, exponent_loglike-np.max(exponent_loglike), label=name)
    # print(len(exponent_loglike))
    # plt.axvline(exponent_grid[np.argmax(exponent_loglike)], color='black', linestyle='--')
plt.ylim(-60, 5)
plt.legend(frameon=False)
plt.show()

In [ ]:
plt.plot(exponent_grid, np.sum(np.array(exponent_loglikes['pdg2025-stat']), axis=0))

In [ ]:
plt.scatter(re_loglikes['pdg2025-both'], np.array(exponent_loglikes['pdg2025-both'])[:,0], s=3)

In [ ]:
which = 'pdg2025-both'
diffs = np.array(exponent_loglikes[which])[:,0]-re_loglikes[which]
print(np.sum(diffs))
bad_idx = np.argmax(diffs)

name = names[which][bad_idx]
print(name)
for df in datasets[which]:
    if df['pdgid'].iloc[0] == name:
        break

values = np.array(df['value'])
sigmas = np.array(df['uncertainty'])
print(values, sigmas)
_, theta_mle, _, tau_mle = random_effects_mle(values, sigmas, coverage=0.6827)
print(theta_mle, tau_mle)
tau_grid = np.concatenate((np.logspace(-4,4,200)*max(sigmas), np.logspace(-4,4,200)))
theta_grid = np.concatenate((np.linspace(muhat_birge, muhat_re, 100), np.linspace(np.min(values), np.max(values), 100)))
scale = np.sqrt(sigmas[:,np.newaxis,np.newaxis]**2 + (tau_grid)**2)
loglike = np.sum(norm.logpdf(values[:,np.newaxis,np.newaxis], loc=theta_grid[:,np.newaxis], scale=scale), axis=0)

theta_idx, tau_idx = np.unravel_index(np.argmax(loglike), loglike.shape)
print(theta_grid[theta_idx], tau_grid[tau_idx])

print(np.sum(norm.logpdf(values, loc=theta_mle, scale=np.sqrt(sigmas**2+tau_mle**2))))
print(np.sum(norm.logpdf(values, loc=theta_grid[theta_idx], scale=np.sqrt(sigmas**2+tau_grid[tau_idx]**2))))

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(8, 3), sharey=True, gridspec_kw={'wspace': 0.1})
for i, data in enumerate([ns, I2s, brs]):
    parts = axs[i].violinplot(list(data.values()), vert=False, showmedians=False, showextrema=False)
    axs[i].tick_params(top=True, right=True)
    for j in range(len(data)):
        axs[i].axhline(j+1, color='grey', linewidth=0.5, linestyle=':')
    for pc in parts['bodies']:
        pc.set_facecolor('grey')
        pc.set_edgecolor('none')
        pc.set_alpha(1)
    # add median
    medians = [np.median(data[name]) for name in data.keys()]
    print(medians)
    axs[i].scatter(medians, np.array(range(len(data)))+1, marker='|', s=50, c='black', alpha=1, linewidths=1)

axs[0].set_xlabel('Number of results')
axs[1].set_xlabel('$I^2$')
axs[2].set_xlabel('Birge Ratio')

# add y ticks
axs[0].set_yticks(np.array(range(len(datasets)))+1)
axs[0].set_yticklabels([nice_names_break[name] for name in datasets.keys()])
# flip y axis
axs[0].invert_yaxis()
axs[2].set_xlim(1, 5)
axs[0].set_xlim(0, None)
# add right ticks
axs[0].tick_params(top=True, right=True)
axs[1].tick_params(top=True, right=True)
axs[2].tick_params(top=True, right=True)
axs[1].set_xlim(0, 1)

plt.tight_layout()
plt.savefig('figs/noground_violin.pdf', bbox_inches='tight')
plt.show()

In [ ]:
rows = []
rows_numeric = []

def format_number(x):
    if x == -np.inf:
        return r'$\approx-\infty$'
    if x == np.inf:
        return r'$\approx\infty$'
    if x >= 1e3 or x <= -1e3:
        return f'${str(int(x))}$'
    return r'${0:.3g}$'.format(x)

def add_bold(string):
    return r'$\mathbf{' + string.strip('$') + r'}$'

for name in datasets.keys():
    loglikes = np.array([birge_loglikes[name], re_loglikes[name], fe_loglikes[name]])
    total_loglikes = np.sum(loglikes, axis=1)
    assert len(total_loglikes) == 3
    ks = np.array([2, 2, 1])
    
    bics = ks[:, np.newaxis] * np.log(np.array(ns[name])) - 2 * loglikes
    aics = 2 * ks[:, np.newaxis] - 2 * loglikes
    bics = np.mean(bics, axis=1)
    aics = np.mean(aics, axis=1)
    numbers = list(-total_loglikes) + list(bics) + list(aics)
    numbers_str = [format_number(x) for x in numbers]
    rows.append([r'\texttt{' + name + r'}'] + numbers_str)
    rows_numeric.append(numbers)
rows_numeric = np.array(rows_numeric)

best_loglike = np.argmin(rows_numeric[:, :3], axis=1)
best_bic = np.argmin(rows_numeric[:, 3:6], axis=1)
best_aic = np.argmin(rows_numeric[:, 6:], axis=1)

# add bold to the best model
for i, row in enumerate(rows):
    row[best_loglike[i]+1] = add_bold(row[best_loglike[i]+1])
    row[best_bic[i]+4] = add_bold(row[best_bic[i]+4])
    row[best_aic[i]+7] = add_bold(row[best_aic[i]+7])

# use np to save latex table with & separator
txt = ' \\\\\n'.join([' & '.join(row) for row in rows])
print(txt)
with open('tables/noground-loglike.tex', 'w') as f:
    f.write(txt)

In [ ]:
maxn = np.max([np.max(n) for n in ns.values()])

In [ ]:
fig, axs = plt.subplots(4, 2, figsize=(7, 9), sharex=False, sharey=False, gridspec_kw={'wspace': 0, 'hspace': 0.1})

for i, (name, dataset) in enumerate(datasets.items()):
    ax = axs.flatten()[i]

    sc = ax.scatter(birge_loglikes[name], re_loglikes[name], marker='x', s=10, c=ns[name], vmin=0, vmax=maxn, facecolor='none', linewidths=0.5)
    ymin = np.min([np.min(birge_loglikes[name]), np.min(re_loglikes[name])])
    ymax = np.max([np.max(birge_loglikes[name]), np.max(re_loglikes[name])])
    ax.plot([ymin-1, ymax+1], [ymin-1, ymax+1], color='red', linewidth=1, linestyle=':')
    # set aspect ratio to 1
    ax.set_aspect('equal', adjustable='box')
    ax.set_xlim(ymin-0.2, ymax+0.2)
    ax.set_ylim(ymin-0.2, ymax+0.2)
    ax.set_xlabel('BR log-likelihood')
    ax.set_ylabel('RE log-likelihood')
    # plt.colorbar(label='Number of measurements')
    re_better_count = np.sum(re_loglikes[name] > birge_loglikes[name])
    birge_better_count = np.sum(birge_loglikes[name] > re_loglikes[name])
    total_count = len(birge_loglikes[name])
    re_better_frac = f'{re_better_count}/{total_count}'
    birge_better_frac = f'{birge_better_count}/{total_count}'
    ax.text(0.3, 0.5, f'RE better ({re_better_frac})', fontsize=8, fontweight='bold', color='black', ha='center', va='center', transform=ax.transAxes, rotation=45)
    ax.text(0.5, 0.3, f'BR better ({birge_better_frac})', fontsize=8, fontweight='bold', color='black', ha='center', va='center', transform=ax.transAxes, rotation=45)
    ax.text(0.05, 0.95, nice_names_break[name], transform=ax.transAxes, ha='left', va='top')
fig.suptitle('Random Effects and Birge Ratio MLE log-likelihoods for each set of results')
axs.flatten()[-1].axis('off')
plt.tight_layout()
fig.subplots_adjust(right=0.85)
cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
fig.colorbar(sc, cax=cbar_ax, label='Number of measurements')
fig.subplots_adjust(top=0.95)
plt.savefig('figs/noground_loglike.pdf', bbox_inches='tight')
plt.show()


In [ ]:
plt.figure(figsize=(8,3))
for i, (name, dataset) in enumerate(datasets.items()):
    diff = re_loglikes[name] - birge_loglikes[name]
    plt.scatter(diff, -np.ones(len(diff))*i, marker='|', s=20, vmin=0, vmax=maxn, facecolor='none', linewidths=0.4, c='black', alpha=0.5)
    plt.axhline(-i, color='grey', linewidth=0.3, linestyle=':')

    re_better_count = np.sum(re_loglikes[name] > birge_loglikes[name])
    birge_better_count = np.sum(birge_loglikes[name] > re_loglikes[name])
    total_count = len(birge_loglikes[name])
    re_better_frac = f'{re_better_count}/{total_count}'
    birge_better_frac = f'{birge_better_count}/{total_count}'
    plt.text(1, -i+0.3, f'RE better: {re_better_frac}', fontsize=8, fontweight='bold', color='black', ha='left', va='center')
    plt.text(-1, -i+0.3, f'BR better: {birge_better_frac}', fontsize=8, fontweight='bold', color='black', ha='right', va='center')
    
plt.xlabel('RE log-likelihood - BR log-likelihood')
plt.ylabel('Set of results')
plt.yticks(range(0, -len(datasets), -1), [nice_names_break[name] for name in datasets.keys()])
plt.tight_layout()
plt.axvline(0, color='red', linewidth=1, linestyle=':')
xmax = max(np.abs(np.array(plt.xlim())))
plt.xlim(-xmax, xmax)
plt.ylim(-len(datasets)+0.5, 0.8)
plt.gca().tick_params(top=True)
plt.savefig('figs/noground_loglike_diff.pdf', bbox_inches='tight')
plt.show()